In [ ]:
"""Rotate Spin on x-y Plane"""
%matplotlib
import time
import kwant
import numpy as np
import scipy.linalg as sla
import scipy.sparse
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tinyarray

s_x = tinyarray.array([[0, 1], [1, 0]])
s_y = tinyarray.array([[0, -1j], [1j, 0]])
s_z = tinyarray.array([[1, 0], [0, -1]])
s_0 = tinyarray.array([[1, 0], [0, 1]])
sigma_x = tinyarray.array([[0, 1], [1, 0]])
sigma_y = tinyarray.array([[0, -1j], [1j, 0]])
sigma_z = tinyarray.array([[1, 0], [0, -1]])
sigma_0 = tinyarray.array([[1, 0], [0, 1]])
tau_x = tinyarray.array([[0, 1], [1, 0]])
tau_y = tinyarray.array([[0, -1j], [1j, 0]])
tau_z = tinyarray.array([[1, 0], [0, -1]])
tau_0 = tinyarray.array([[1, 0], [0, 1]])


'''
BHZ model
L and W: TI length and width
L2 and L1: SC width and FI width
A,B,m : constant parameters in BHZ model
mu: chemical potential 
F: spin of FI
theda and phi: polar and azimuthal angle of FI spin
SOC is in z direction and TI is on x-y plane
'''


def make_sys(a=1, A=1, B=1, mu=0.474,  m=1.6,L=50, W=15, L1=5, L2=20, delta=1, F=1, theta=np.pi * 2 / 4):

    def onsite_S(site, phi):
        return (m - 4 * B) * np.kron(tau_z, np.kron(s_0, sigma_z)) + delta * \
               np.kron(1j * tau_y, np.kron(1j * s_y,sigma_0)) - mu * np.kron(tau_z, np.kron(s_0, sigma_0))

    def onsite_Fup(site, phi):
        return (m - 4 * B) * np.kron(tau_z, np.kron(s_0, sigma_z)) + F * np.sin(theta) * np.cos(phi) *\
               np.kron(tau_z, np.kron(s_x, sigma_0)) + F * np.sin(theta) * np.sin(phi) * \
               np.kron(tau_0, np.kron(s_y, sigma_0)) + F * np.cos(theta) * np.kron(tau_z, np.kron(s_z, sigma_0)) \
               - mu * np.kron(tau_z, np.kron(s_0, sigma_0))

    def onsite_Fdown(site, phi):
        return (m - 4 * B) * np.kron(tau_z, np.kron(s_0, sigma_z)) + F * np.sin(theta) * np.cos(phi) \
               * np.kron(tau_z, np.kron(s_x, sigma_0)) + F * np.sin(theta) * np.sin(phi) *\
               np.kron(tau_0, np.kron(s_y, sigma_0)) + F * np.cos(theta) * np.kron(tau_z, np.kron(s_z, sigma_0)) \
               - mu * np.kron(tau_z, np.kron(s_0, sigma_0))

    def onsite(site, phi):
        return (m - 4 * B) * np.kron(tau_z, np.kron(s_0, sigma_z)) - mu * np.kron(tau_z, np.kron(s_0, sigma_0))

    def hoppingx(site1, site2, phi):
        return B * np.kron(tau_z, np.kron(s_0, sigma_z)) + 1j * A * np.kron(tau_0, np.kron(s_z, sigma_x))

    def hoppingy(site1, site2, phi):
        return B * np.kron(tau_z, np.kron(s_0, sigma_z)) - 1j * A * np.kron(tau_z, np.kron(s_0, sigma_y))

    lat = kwant.lattice.square(a)
    sys = kwant.builder.Builder()
    sys[(lat(x, y) for x in range(L) for y in range(W))] = onsite
    sys[(lat(x, y) for x in range(L2) for y in range(W))] = onsite_S
    sys[(lat(x, y) for x in range(L2, L - L2)for y in range(L1))] = onsite_Fdown
    sys[(lat(x, y) for x in range(L2, L - L2) for y in range(W - L1, W))] = onsite_Fup
    sys[(lat(x, y) for x in range(L - L2, L) for y in range(W))] = onsite_S
    sys[kwant.builder.HoppingKind((1, 0), lat, lat)] = hoppingx
    sys[kwant.builder.HoppingKind((0, 1), lat, lat)] = hoppingy
    return sys


'''
calculate the electron density of state of one of splitted MZM

'''
def sel(wave, sys):
    waves = wave * np.conj(wave)
    Wave = np.empty((15, 50), dtype='complex64')
    for j in range(4):
        wave1 = waves[j::8]
        for i in range(len(sys.sites)):
            x, y = sys.sites[i].pos
            Wave[int(y), int(x)] += wave1[i]
    return Wave




'''
function used to split the four MZM.
if you change the size of the system , the position of MZM is changed.
So x, y parameters below need to be modified.
'''
def calco(waves,sys,L=50, W=15, L1=5, L2=20):
    A = np.empty((4, 4), dtype='complex64')
    X = np.empty((4, 4), dtype='complex64')
    B = np.empty((4, 4), dtype='complex64')
    C = np.empty((4, 4), dtype='complex64')
    Waves = np.empty((len(sys.sites) * 8, 4), dtype='complex64')
    for i in range(4):
        wave = waves[:, i]
        waveB = wave[::8]
        for j in range(len(sys.sites)):
            x, y = sys.sites[j].pos
            if x == L2-1 and y == 0:
                B[i, 0] = np.abs(waveB[j])
                C[i, 0] = waveB[j] / np.abs(waveB[j])
            elif x == L-L2 and y == 0:
                B[i, 1] = np.abs(waveB[j])
                C[i, 1] = waveB[j] / np.abs(waveB[j])
            elif x == L2-1 and y == W-1:
                B[i, 2] = np.abs(waveB[j])
                C[i, 2] = waveB[j] / np.abs(waveB[j])
            elif x == L-L2 and y == W-1:
                B[i, 3] = np.abs(waveB[j])
                C[i, 3] = waveB[j] / np.abs(waveB[j])
    for j in range(4):
        for i in range(4):
            A[i, j] = B[i, j] / B[3, j]
    for j in range(4):
        for i in range(4):
            X[i, j] = B[3, j] * A[i, j] * 1 / np.sqrt(A[0, j] ** 2 + A[1, j] ** 2 + A[2, j] ** 2 + A[3, j] ** 2)
    for j in range(4):
        Waves[:, j] = X[0, j] * waves[:, 0] / C[0, j] + X[1, j] * waves[:, 1] / C[1, j] + X[2, j] * waves[:, 2] / C[2, j] + X[3, j] * waves[:, 3] / C[3, j]
    return Waves



'''
plot the distribution of DOS for one MZM

'''
def plot(sys):
    ham_mat = sys.hamiltonian_submatrix(args=[np.pi * 5 / 4], sparse=True)
    value, vec = scipy.sparse.linalg.eigsh(scipy.sparse.csc_matrix(ham_mat), k=20, sigma=1e-9)
    wave = calco(vec[:, :4], sys)
    wave1 = sel(wave[:,0], sys)
    print(value[:12])
    x = np.linspace(0, 49, 50)
    y = np.linspace(0, 14, 15)
    X, Y = np.meshgrid(x, y)
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.plot_surface(X, Y, wave1.real, rstride=1, cstride=1)
    plt.show()
    return wave[:, 0]


'''
check the self-hermitian of MZM during the evolution

'''
def main():
    '''
    Add a global phase to the MZM wavefunction to make it self-hermitian.
    
    '''
    system = make_sys().finalized()
    kwant.plot(system)
    majorana_wave = plot(system)
    index = np.argmax(np.abs(majorana_wave[1::8]))
    phase = np.sqrt(np.conj(majorana_wave[5::8][index]) / majorana_wave[1::8][index])
    majorana_wave *= phase
    print(" the beginning self hermitian:", np.abs(majorana_wave[1::8] - np.conj(majorana_wave[5::8]))[index])

    '''
    rotate the spin of FI for a round with 200 steps with time interval 0.5s. And check its  self-hermitian.
    
    '''
    
    tstep, dt = 200, 0.5
    for i in range(tstep):
        print(i)
        hamiltonian = system.hamiltonian_submatrix(args=[np.pi * 5 / 4 + 2 * np.pi * (i + 1) / tstep], sparse=False)
        energies, states = sla.eigh(hamiltonian)
        sizeH = len(energies)
        unitary = states.dot(np.diag(np.exp(-1j * dt * energies))).dot(np.matrix.getH(states))
#         print("the determination of unitary operator:",np.linalg.det(unitary))
        majorana_wave = unitary.dot(majorana_wave)
        print("the eigenvalue of MZM:", energies[sizeH // 2])
        print("self hermitian:", np.abs(majorana_wave[1::8] - np.conj(majorana_wave[5::8]))[index])
if __name__ == '__main__':
    main()



Using matplotlib backend: TkAgg


c:\users\zhangsuqi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:139: KwantDeprecationWarning: The 'args' parameter has been deprecated since version 1.4 -- Instead, provide named parameters as a dictionary via 'params'.


[ 0.00922699  0.00921116 -0.00921116 -0.00922699  0.31863614  0.31868316
 -0.31863614 -0.31868316  0.52873945  0.52855871 -0.52855871 -0.52873945]
 the beginning self hermitian: 1.4167205e-07
0


c:\users\zhangsuqi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:179: KwantDeprecationWarning: The 'args' parameter has been deprecated since version 1.4 -- Instead, provide named parameters as a dictionary via 'params'.


the eigenvalue of MZM: 0.009211162504982755
self hermitian: 5.412722582672816e-07
1
the eigenvalue of MZM: 0.009211162504982813
self hermitian: 1.0451008748146104e-06
2
the eigenvalue of MZM: 0.009211162504983095
self hermitian: 1.553151325027324e-06
3
the eigenvalue of MZM: 0.00921116250498263
self hermitian: 2.0674059120903324e-06
4
the eigenvalue of MZM: 0.009211162504982766
self hermitian: 2.589319239949362e-06
5
the eigenvalue of MZM: 0.009211162504983267
self hermitian: 3.1213013110922024e-06
6
the eigenvalue of MZM: 0.009211162504983238
self hermitian: 3.6727774045353235e-06
7
the eigenvalue of MZM: 0.009211162504983193
self hermitian: 4.260627275631423e-06
8
the eigenvalue of MZM: 0.009211162504983179
self hermitian: 4.8977025596703285e-06
9
the eigenvalue of MZM: 0.009211162504983115
self hermitian: 5.585192277123657e-06
10
the eigenvalue of MZM: 0.009211162504982738
self hermitian: 6.278767066513864e-06
11
the eigenvalue of MZM: 0.009211162504982955
self hermitian: 6.91730419